## check keras backend to see if GPU is running

In [1]:
import keras
keras.backend.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

## data pre-process
### data visulization

In [2]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

training_image_path = 'dataset/images_prepped_train'
training_label_path = 'dataset/annotations_prepped_train'
trainingFileNameList = os.listdir(training_image_path)
# img_training = mpimg.imread(os.path.join(training_image_path,trainingFileNameList[0]))
img_training = cv2.imread(os.path.join(training_image_path,trainingFileNameList[10]))
labelingFileNameList = os.listdir(training_label_path)
img_labeling = cv2.imread(os.path.join(training_label_path,labelingFileNameList[10]))

rows, cols, chanels = img_labeling.shape

mi, ma = np.min(img_labeling), np.max(img_labeling)
n_classes = ma - mi + 1
print('Number of Classes in this label image:{},image hight:{},image width:{}'.format(n_classes, rows, cols))
print('minimum label img = {}, maximum label img = {}, Total number of label img classes = {}'.format(mi, ma, n_classes))
img_training = cv2.resize(img_training,((cols//32)*32, (rows//32)*32))
img_labeling = cv2.resize(img_labeling,((cols//32)*32, (rows//32)*32))
# Plotting
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title('training BGR')
ax1.imshow(img_training)

ax2.set_title('labeling')
ax2.imshow(img_labeling)

fig = plt.figure(figsize=(15,10))
for k in range(mi,ma+1):
    ax = fig.add_subplot(3,n_classes/3,k+1)
    ax.imshow((img_labeling == k)*1.0)
    ax.set_title("label = {}".format(k-mi))
    
plt.show()

Number of Classes in this label image:12,image hight:360,image width:480
minimum label img = 0, maximum label img = 11, Total number of label img classes = 12


<Figure size 640x480 with 2 Axes>

<Figure size 1500x1000 with 12 Axes>

### data prepare

In [3]:
from sklearn.model_selection import train_test_split
import sklearn
def generator(samples, batch_size = 32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
#         shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            labels = []
            for batch_sample in batch_samples:
                trainPath = os.path.join(training_image_path, batch_sample)
                labelPath = os.path.join(training_label_path, batch_sample)
                img_train = cv2.imread(trainPath)
                img_train = cv2.resize(img_train, ((cols//32)*32, (rows//32)*32))
                
                img_label = cv2.imread(labelPath)
                img_label = cv2.resize(img_label, ((cols//32)*32, (rows//32)*32))
                seg_labels = np.zeros(((rows//32)*32 , (cols//32)*32  , n_classes ))
                img_label = img_label[:, : , 0]

                for c in range(n_classes):
                    seg_labels[: , : , c ] = (img_label == c ).astype(int)
        
                images.append(img_train)
                labels.append(seg_labels)

            x = np.array(images)
            y = np.array(labels)
            yield sklearn.utils.shuffle(x, y)
            

# train_generator = generator(trainingFileNameList, batch_size=32)
train_samples, validation_samples = train_test_split(trainingFileNameList, test_size=0.2)
train_generator = generator(train_samples, batch_size=8)
validation_generator = generator(validation_samples, batch_size=8)

In [4]:
# from tqdm import tqdm
# import numpy as np
# trainingImgs = []
# labelImgs = []
# for file in tqdm(trainingFileNameList):
#     path = os.path.join(training_image_path, file)
#     img = cv2.imread(path, 1)
#     img_resize = cv2.resize(img, ((cols//32)*32, (rows//32)*32))
#     trainingImgs.append(img_resize)
    
# for file in tqdm(labelingFileNameList):
#     path = os.path.join(training_label_path, file)
#     img = cv2.imread(path, 1)
#     img_resize = cv2.resize(img, ((cols//32)*32, (rows//32)*32))
    
    
#     seg_labels = np.zeros(((rows//32)*32 , (cols//32)*32  , n_classes ))
#     img_resize = img_resize[:, : , 0]
    
#     for c in range(n_classes):
#         seg_labels[: , : , c ] = (img_resize == c ).astype(int)
#     ##seg_labels = np.reshape(seg_labels, ( width*height,nClasses  ))
#     labelImgs.append(seg_labels)
    
# assert len(trainingImgs) == len(labelImgs), 'training set and labeling set should be same size.'
# trainingImgs = np.array(trainingImgs)
# labelImgs = np.array(labelImgs)
# print('training image set size:{}, label image set size:{}'.format(trainingImgs.shape, labelImgs.shape))

## Import Keras and Tensorflow to develop deep learning FCN models

In [5]:
# location of VGG weights
VGG_Weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [6]:
from keras.models import *
from keras.layers import *
def FCN8( nClasses, input_height=224, input_width=224):
    ## input_height and width must be devisible by 32 because maxpooling with filter size = (2,2) is operated 5 times,
    ## which makes the input_height and width 2^5 = 32 times smaller
    assert input_height%32 == 0, 'input_height must be devisible by 32 because maxpooling with filter size = (2,2)'
    assert input_width%32 == 0, 'input_width must be devisible by 32 because maxpooling with filter size = (2,2)'
    IMAGE_ORDERING =  "channels_last" 

    img_input = Input(shape=(input_height,input_width, 3)) ## image shape
    
    ## Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
    f1 = x
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
    pool3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)## (None, 14, 14, 512) 

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(pool4)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)## (None, 7, 7, 512)

    #x = Flatten(name='flatten')(x)
    #x = Dense(4096, activation='relu', name='fc1')(x)
    # <--> o = ( Conv2D( 4096 , ( 7 , 7 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)
    # assuming that the input_height = input_width = 224 as in VGG data
    
    #x = Dense(4096, activation='relu', name='fc2')(x)
    # <--> o = ( Conv2D( 4096 , ( 1 , 1 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)   
    # assuming that the input_height = input_width = 224 as in VGG data
    
    #x = Dense(1000 , activation='softmax', name='predictions')(x)
    # <--> o = ( Conv2D( nClasses ,  ( 1 , 1 ) ,kernel_initializer='he_normal' , data_format=IMAGE_ORDERING))(o)
    # assuming that the input_height = input_width = 224 as in VGG data
    
    
    vgg  = Model(  img_input , pool5  )
    vgg.load_weights(VGG_Weights_path) ## loading VGG weights for the encoder parts of FCN8
    
    n = 4096
    o = ( Conv2D( n , ( 7 , 7 ) , activation='relu' , padding='same', name="conv6", data_format=IMAGE_ORDERING))(pool5)
    conv7 = ( Conv2D( n , ( 1 , 1 ) , activation='relu' , padding='same', name="conv7", data_format=IMAGE_ORDERING))(o)
    
    
    ## 4 times upsamping for pool4 layer
    conv7_4 = Conv2DTranspose( nClasses , kernel_size=(4,4) ,  strides=(4,4) , use_bias=False, data_format=IMAGE_ORDERING )(conv7)
    ## (None, 224, 224, 10)
    ## 2 times upsampling for pool411
    pool411 = ( Conv2D( nClasses , ( 1 , 1 ) , activation='relu' , padding='same', name="pool4_11", data_format=IMAGE_ORDERING))(pool4)
    pool411_2 = (Conv2DTranspose( nClasses , kernel_size=(2,2) ,  strides=(2,2) , use_bias=False, data_format=IMAGE_ORDERING ))(pool411)
    
    pool311 = ( Conv2D( nClasses , ( 1 , 1 ) , activation='relu' , padding='same', name="pool3_11", data_format=IMAGE_ORDERING))(pool3)
        
    o = Add(name="add")([pool411_2, pool311, conv7_4 ])
    o = Conv2DTranspose( nClasses , kernel_size=(8,8) ,  strides=(8,8) , use_bias=False, data_format=IMAGE_ORDERING )(o)
    o = (Activation('softmax'))(o)
    
    model = Model(img_input, o)

    return model

model = FCN8(nClasses = n_classes,  
             input_height = (rows//32)*32, 
             input_width  = (cols//32)*32)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 352, 480, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 352, 480, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 352, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 176, 240, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

## Split between training and validation data

## Train FCN model

In [7]:
from keras import optimizers
sgd = optimizers.SGD(lr=1E-2, decay=5**(-4), momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# model.fit(trainingImgs, labelImgs, validation_split=0.1, shuffle=True, batch_size=32, nb_epoch=5)

model.fit_generator(train_generator, 
                    samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), 
                    nb_epoch=3)

C:\Users\qwang97\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  del sys.path[0]
C:\Users\qwang97\AppData\Local\Continuum\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=3, steps_per_epoch=293, validation_data=<generator..., validation_steps=74)`
  del sys.path[0]


Epoch 1/3
293/293 [==============================] - 405s 1s/step - loss: 13.9639 - acc: 0.1309 - val_loss: 13.9702 - val_acc: 0.1333
Epoch 2/3
293/293 [==============================] - 391s 1s/step - loss: 13.9610 - acc: 0.1338 - val_loss: 13.9704 - val_acc: 0.1332
Epoch 3/3
293/293 [==============================] - 391s 1s/step - loss: 13.9609 - acc: 0.1338 - val_loss: 13.9729 - val_acc: 0.1331
